# An Explain Report

This is a POC for an explain report: given a number at time T1 and time T2, introspect the clocks involved to produce a simple linear breakdown
of what changed on various timelines to account for the change in the number between T1 and T2.

The report is an ordered sequence of clock changes: the sum of changes should equal to the total change. The order of application is important: change the
ordering, and the resultant values may change. It's not a Jacobian-type report with T1 deltas and no ordering.

The real reason for this workbook is to start working on the node class optimizations and metadata management.

## Footnotes

Footnotes are one of the simplest types of metadata:
* Any computation (even a simple read of a constant value) may declare it has footnotes
* Every computation gets the footnote of all its inputs

Footnoting is based on a number of observations about large software systems (say, 1M LOC or more):
* Any complex report (i.e. computation output) will generally be somewhat wrong, misleading, or out of date.
* If you want 100% correctness and truth, your report will raise an exception every time you run it.
* If you just want it to run, you'll get a result, but some log file on some machine in some compute farm will have a message explaining why your result is wrong.
* And you won't see that message.
* And that message may have leaked information that bad actors can read.
* And you won't trust this system, so your own developers will copy the underlying data and write their own version of the report with all the same issues.
* Now you don't have a problem, but your company now has two problems.

Footnotes summary:
* Are the documentation of problems from the producer's point of view
* Make no claim about usability of results, etc, for a specific consumer
* May be programmatically removed/condensed/replaced at controlled code points


In [1]:
from mand.core import Entity, node, Context
from mand.core import ObjectDb, Timestamp, getNode, _tr
from mand.core import ProfileMonitor
from mand.core import displayDict, displayMarkdown, displayListOfDicts, displayHeader
from mand.core import find
from mand.demos.trading import makeWorld, bookSomeTrades

db = ObjectDb()

from mand.lib.dbsetup import setUpDb
setUpDb(db)
db.describe()

<mand.db.ObjectDb object at 0x10f166ef0>: 31, mem=True, ro=False: entities=9, map=2


In [2]:
with db:
    pWorld = makeWorld()
    
pAll, bExt, bExt2 = pWorld.children()
p1 = pAll.children()[0]
p2 = pAll.children()[1]
p4 = pAll.children()[3]

b2 = p2.children()[0]
b4 = p4.children()[0]

makeWorld, TopOfTheHouse is: <Entity:/Global/TradingPortfolio/TopOfTheHouse>
    # books: 100
    # children: 10


# Book some trades

This time, we throw a few thousand in...

In [3]:
ts0, ts1, ts2, ts3, ts4, ts5, eod, ts6 = bookSomeTrades(pWorld)

# An Explain Report

This is a mix of abstractions at the Core, DBA, and BA, and User levels. But, it does the job for now...

In [4]:
class Report(Entity):
    @node(stored=True)
    def valuable(self):
        return None
    
    @node(stored=True)
    def ts1(self):
        return None
    
    @node(stored=True)
    def ts2(self):
        return None
    
    @node
    def cutoffs(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        
        clock = valuable.getObj(_tr.RootClock, 'Main')
        
        def cuts(ts):
            def fn(node):
                obj = node.key.object()
                m = node.key.shortName()
                if isinstance(obj, _tr.Clock) and m == 'cutoffs':
                    return True
            with Context({clock.cutoffs: ts}, 'Clocks'):
                nodes = find(valuable.NPV, fn)
                return nodes

        ret = {}
        for n in cuts(ts1):
            ret[n.key._key] = n
        for n in cuts(ts2):
            ret[n.key._key] = n
        return ret.values()
    
    @node
    def data(self):
        valuable = self.valuable()
        ts1 = self.ts1()
        ts2 = self.ts2()
        clock = valuable.getObj(_tr.RootClock, 'Main')
    
        nodes = self.cutoffs()
    
        # IRL, we'd sort these according to some business req...
        # And our clocks might be arranged in an N-level tree...
        nodes = sorted(nodes, key = lambda node: node.object().meta.name())
    
        data = []
        curr = [0]
        def add(title, npv):
            pnl = npv - curr[0]
            curr[0] = npv
            data.append( {'Activity': title, 'PnL': pnl } )

        with Context({clock.cutoffs: ts1}, 'Start'):
            curr = [ valuable.NPV() ] # Starting balance
    
        tweaks = {}
        for n in nodes:
            tweaks[n] = ts1
        with Context(tweaks, name='Start breaks'):
            start = valuable.NPV()
            add('Starting balance breaks', start)

        tsAmend = Timestamp(t=ts2.transactionTime, v=ts1.validTime)
        # XXX - modifying tweaks in place is a bit evil
        # This is only safe because I know Context() effectively copies, so this works
        # for now.
        for n in nodes:
            tweaks[n] = tsAmend
            name = n.object().meta.name()
            with Context(tweaks, name='Amend %s' % name):
                add('prior day amends: %s' % name, valuable.NPV())
        for n in nodes:
            tweaks[n] = ts2
            name = n.object().meta.name()
            with Context(tweaks, name='Activity %s' % name):
                add('activity: %s' % name, valuable.NPV())
    
        with Context({clock.cutoffs: ts2}, name='End'):
            end = valuable.NPV()
            add('Ending balance breaks', end)

        title = 'PnL explain for %s: %s' % (valuable.meta.name(), end-start)
        return data, title

    def run(self):
        data, title = self.data()
        node = getNode(self.data)
        footnotes = node.footnotes.values()
        displayHeader('%s' % title)
        if footnotes:
            displayMarkdown('**Caveat: this report encountered problems. See footnotes at bottom.**')
        displayListOfDicts(data, names=['Activity', 'PnL'] )
        if footnotes:
            displayMarkdown('## Footnotes:')
            displayMarkdown(node.footnoteMarkdown())
    
r = Report(valuable=pAll, ts1=eod, ts2=ts6)
r.run()

1. Inadequate cash discounting model used

# PnL explain for TopOfTheHouse: 5236.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|0.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used

## Add some inconsistent data [Test]

Book b2 should appear multiple times in some portfolio trees and be flagged accordingly...

In [5]:
with db:
    p1.setChildren(p1.children() + [b2])

ts7 = Timestamp()

# Footnotes

Note the report calculation has run, but attached appropriate caveats to the output:

In [6]:
db3 = db.copy()
p = db3._get(pAll.meta.path())

with ProfileMonitor(mode='sum'): 
    r = Report(valuable=p, ts1=eod, ts2=ts7)
    r.run()

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0

# PnL explain for TopOfTheHouse: 6256.00

**Caveat: this report encountered problems. See footnotes at bottom.**

|Activity|PnL|
|-|-|
|Starting balance breaks|0.00
|prior day amends: MarketData|0.00
|prior day amends: Portfolio|0.00
|prior day amends: Trading|-4.00
|activity: MarketData|5240.00
|activity: Portfolio|1020.00
|activity: Trading|0.00
|Ending balance breaks|0.00

## Footnotes:

1. Inadequate cash discounting model used
1. Book appears multiple times
  * /Global/TradingBook/Eq-Inst0


### Profile by nodes.
* times are in microseconds
* cumT is total time spent in function
* calcT is time spent in function, but not in a child node

|fn|n|cumT|calcT|cumT/call|sys|
|-|-|-|-|-|-|
|Report:data|1|19,731,587|15|19,731,587|GetValue
|Report:data|1|19,731,571|767|19,731,571|GetValue/Calc
|TradingContainer:NPV|11|19,663,560|150|1,787,596|GetValue
|TradingContainer:NPV|11|19,663,410|79|1,787,582|GetValue/Calc
|Portfolio:items|121|19,480,417|1,207|160,995|GetValue
|Portfolio:items|121|19,480,316|838|160,994|GetValue/Calc
|Workbook:items|1,104|18,363,622|12,165|16,633|GetValue
|Workbook:items|1,100|18,351,457|8,250|16,683|GetValue/Calc
|Report:cutoffs|1|11,956,687|20|11,956,687|GetValue
|Report:cutoffs|1|11,956,667|177|11,956,667|GetValue/Calc
|Root:Clocks|548|11,956,478|1,663,885|21,818|Context
|TradeOpenEvent|1,014|5,050,171|5,050,171|4,980|Db.Get
|TradeOpenEvent:ticket|22,286|4,580,781|111,239|205|GetValue
|TradingTicket|1,014|4,469,542|4,469,542|4,407|Db.Get
|Root:Activity Portfolio|263|1,089,699|1,023,085|4,143|Context
|Portfolio:children|242|1,076,390|1,641|4,447|GetValue
|Portfolio:children|121|1,074,748|882|8,882|GetValue/Calc
|Root:Amend MarketData|255|992,368|927,729|3,891|Context
|Root:Start breaks|255|893,473|821,076|3,503|Context
|Root:Start|270|884,315|812,367|3,275|Context
|Root:End|278|834,408|765,904|3,001|Context
|Root:Activity MarketData|255|789,716|724,508|3,096|Context
|Root:Amend Portfolio|255|781,742|717,952|3,065|Context
|Root:Amend Trading|255|753,808|691,433|2,956|Context
|Root:Activity Trading|263|748,277|686,890|2,845|Context
|PortfolioUpdateEvent:children|121|453,387|2,853|3,747|GetValue
|TradingBook|102|415,202|415,202|4,070|Db.Get
|Equity:NPV|15|181,704|205|12,113|GetValue
|Equity:NPV|15|181,498|123|12,099|GetValue/Calc
|MarketInterface:spot|15|171,831|185|11,455|GetValue
|MarketInterface:spot|15|171,646|135|11,443|GetValue/Calc
|ExternalRefData:state|15|161,830|231|10,788|GetValue
|ExternalRefData:state|15|161,599|104|10,773|GetValue/Calc
|RefData:state|15|161,101|163|10,740|GetValue
|RefData:state|15|160,937|109|10,729|GetValue/Calc
|Clock:cutoffs|2,480|117,841|8,754|47|GetValue
|Clock:cutoffs|20|109,159|143|5,457|GetValue/Calc
|Clock:parent|20|108,080|196|5,404|GetValue
|Clock:parent|20|107,884|149|5,394|GetValue/Calc
|_WorkItemEvent:book2|22,286|98,950|91,991|4|GetValue
|_WorkItemEvent:book1|22,286|93,435|93,435|4|GetValue
|TradeOpenEvent:quantity|22,286|92,689|92,689|4|GetValue
|_WorkItemEvent:item|11,143|71,594|63,587|6|GetValue
|TradeOpenEvent:premium|11,143|62,000|58,202|5|GetValue
|TradingBook:clock|2,200|59,836|21,744|27|GetValue
|TradeOpenEvent:action|11,143|59,102|59,102|5|GetValue
|PortfolioUpdateEvent|12|58,935|58,935|4,911|Db.Get
|Event:amends|11,340|56,224|56,224|4|GetValue
|TradeOpenEvent:unitPrice|11,143|50,705|50,705|4|GetValue
|TradingPortfolio|10|42,289|42,289|4,228|Db.Get
|RefDataUpdateEvent|9|38,381|38,381|4,264|Db.Get
|TradingBook:clock|1,100|38,092|11,544|34|GetValue/Calc
|Clock|5|19,994|19,994|3,998|Db.Get
|Portfolio:clock|242|11,064|2,835|45|GetValue
|MarketInterface:source|15|8,991|130|599|GetValue
|MarketInterface:source|15|8,861|94|590|GetValue/Calc
|Equity:refdata|15|8,851|148|590|GetValue
|ClockEvent|2|8,743|8,743|4,371|Db.Get
|Equity:refdata|15|8,702|94|580|GetValue/Calc
|Portfolio:clock|121|8,229|1,298|68|GetValue/Calc
|Portfolio:books|231|8,134|1,739|35|GetValue
|MarketInterface|2|8,016|8,016|4,008|Db.Get
|Equity|2|8,007|8,007|4,003|Db.Get
|MarketDataSource|2|7,966|7,966|3,983|Db.Get
|ClockEvent:parent|8|7,642|69|955|GetValue
|Portfolio:books|121|6,743|849|55|GetValue/Calc
|RootClock|1|6,293|6,293|6,293|Db.Get
|MarketDataSource:clock|30|4,497|312|149|GetValue
|MarketDataSource:clock|15|4,184|182|278|GetValue/Calc
|ForwardCashflow|1|3,797|3,797|3,797|Db.Get
|Entity:clock|40|942|405|23|GetValue
|Entity:clock|20|537|201|26|GetValue/Calc
|ForwardCashflow:NPV|11|334|145|30|GetValue
|RefDataUpdateEvent:data|58|301|301|5|GetValue
|MarketInterface:sourceName|15|298|134|19|GetValue
|ForwardCashflow:NPV|11|189|70|17|GetValue/Calc
|MarketInterface:sourceName|15|163|98|10|GetValue/Calc
|RootClock:cutoffs|52|163|163|3|GetValue
|Equity:assetName|15|89|89|5|GetValue
|Report:valuable|2|10|10|5|GetValue
|Report:ts2|2|7|7|3|GetValue
|Report:ts1|2|7|7|3|GetValue